In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import json
import urllib.request
import xml.etree.ElementTree as ET
import plotly.graph_objects as go
import plotly.io as pio
import psycopg2
import seaborn as sns


from datetime import datetime
from pymongo import MongoClient

In [2]:
# Connecting to MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
project_db = mongo_client["projectdb"]

In [3]:
# Reading unemployment-statistics data
unemployment_statistics_url = "https://data.ny.gov/api/views/dh9m-5v4d/rows.json?accessType=DOWNLOAD"
unemployment_statistics_response = urllib.request.urlopen(unemployment_statistics_url)
unemployment_statistics_data = json.loads(unemployment_statistics_response.read())

# inserting into mongoDB
unemployment_statistics_collection = project_db["unemployment_statistics"]
# deleting extising data
unemployment_statistics_collection.drop()

cols = unemployment_statistics_data["meta"]["view"]["columns"]
cols_without_minus_one = filter(lambda val: val["id"] != -1, cols)
unemployment_statistics_cols = list(cols_without_minus_one)

unemployment_statistics_cols_others_len = len(list(filter(lambda val: val["id"] == -1, cols)))

for row in unemployment_statistics_data["data"]:
    db_row = {}
    for col_meta in unemployment_statistics_cols:
        data_value_index = int(col_meta["position"]) + unemployment_statistics_cols_others_len - 1
        db_row[col_meta["fieldName"]] = row[data_value_index]
    unemployment_statistics_collection.insert_one(db_row)

print("Inserted unemployment_statistics document count: ", unemployment_statistics_collection.count_documents({}))

Inserted unemployment_statistics document count:  1689


In [4]:
# Reading covid_cases
covid_cases_url = "https://data.cityofnewyork.us/api/views/rc75-m7u3/rows.json?accessType=DOWNLOAD"
covid_cases_response = urllib.request.urlopen(covid_cases_url)
covid_cases_data = json.loads(covid_cases_response.read())

# inserting into mongoDB
covid_cases_collection = project_db["covid_cases"]
# deleting extising data
covid_cases_collection.drop()


covid_cases_cols = list(filter(lambda val: val["id"] != -1, covid_cases_data["meta"]["view"]["columns"]))
covid_cases_cols_others_len = len(list(filter(lambda val: val["id"] == -1, covid_cases_data["meta"]["view"]["columns"])))


for row in covid_cases_data["data"]:
    db_row = {}
    for col in covid_cases_cols:
        db_row[col["fieldName"].lower()] = row[int(col["position"]) + covid_cases_cols_others_len - 1]
    covid_cases_collection.insert_one(db_row)

print("Inserted covid_cases document count: ",covid_cases_collection.count_documents({}))

Inserted covid_cases document count:  1019


In [5]:
# Reading air_quality
air_quality_url = "https://data.cityofnewyork.us/api/views/c3uy-2p5r/rows.xml?accessType=DOWNLOAD"
air_quality_response = urllib.request.urlopen(air_quality_url)
air_quality_xml = ET.parse(air_quality_response)
air_quality_xml_root = air_quality_xml.getroot()
air_quality_data = []

# inserting into mongoDB
air_quality_collection = project_db["air_quality"]
# deleting extising data
air_quality_collection.drop()

for row in air_quality_xml_root[0].findall('row'):
    air_quality_data_row = {}
    for elem in row.iter():
        if elem is not row:
            key = elem.tag.lower()
            value = elem.text
            #{"key":"value"}             
            air_quality_data_row[key] = value
    if len(air_quality_data_row) != 0:
        air_quality_collection.insert_one(air_quality_data_row)
print("Inserted air_quality document count: ", air_quality_collection.count_documents({}))

Inserted air_quality document count:  16122


In [ ]:
#  MongoDB collections
# 1. unemployment_statistics_collection
# 2. covid_cases_collection
# 3. air_quality_collection

In [ ]:
# creating table in postgress
conn = psycopg2.connect(database="postgres",
                        host="localhost",
                        user="postgres",
                        password="admin",
                        port="5432")

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS unemployment_statistics;CREATE TABLE unemployment_statistics (unemployment_statistics_id serial PRIMARY KEY, date_of_interest TIMESTAMP NOT NULL, area VARCHAR(255), labor_force BIGINT, employed BIGINT, unemployed BIGINT, unemployment_rate REAL);")
cursor.execute("DROP TABLE IF EXISTS covid_cases;CREATE TABLE covid_cases (covid_cases_id serial PRIMARY KEY, date_of_interest TIMESTAMP NOT NULL, case_count INTEGER);")
cursor.execute("DROP TABLE IF EXISTS air_quality;CREATE TABLE air_quality (air_quality_id serial PRIMARY KEY, date_of_interest TIMESTAMP NOT NULL, data_value REAL);")

conn.commit()

In [ ]:
unemployment_statistics_collection = project_db["unemployment_statistics"]
query = """INSERT INTO unemployment_statistics 
    (date_of_interest, area, labor_force, employed, unemployed, unemployment_rate) 
    VALUES (%s,%s,%s,%s,%s,%s)"""
cursor = conn.cursor()
for row in unemployment_statistics_collection.find():
    date = row["year"]+"-"+row["month"]+"-01"
    cursor.execute(query, (date, row["area"], row["labor_force"], row["employed"], 
                           row["unemployed"], row["unemployment_rate"]))
cursor.close()
conn.commit()

In [ ]:
covid_cases_collection = project_db["covid_cases"]

cursor = conn.cursor()
for row in covid_cases_collection.find():
    date = row["date_of_interest"]
    cursor.execute("INSERT INTO covid_cases (date_of_interest, case_count) VALUES (%s,%s)", 
                   (date, row["case_count"]))
cursor.close()
conn.commit()

In [ ]:
air_quality_collection = project_db["air_quality"]

cursor = conn.cursor()
for row in air_quality_collection.find():
    date = row["start_date"]
    cursor.execute("INSERT INTO air_quality (date_of_interest, data_value) VALUES (%s,%s)", 
                   (date, row["data_value"]))
cursor.close()
conn.commit()

In [ ]:
cursor = conn.cursor();
query = """
SELECT date_of_interest, AVG(unemployment_rate)
    FROM unemployment_statistics 
    WHERE date_of_interest >= '2020-01-01' and date_of_interest < '2022-01-01'
    GROUP BY date_of_interest ORDER BY date_of_interest ASC 

"""

    
cursor.execute(query)
unemployment_rate_data = cursor.fetchall()
cursor.close()

# print(unemployment_rate_data)

In [ ]:

unemployment_rate_df = pd.DataFrame(data = unemployment_rate_data,
                                   columns=["Date",
                                            "Unemployment Rate"])

# x_dates = unemployment_rate_df['Date'].dt.strftime('%Y-%m-%d').sort_values().unique()

fig, ax = plt.subplots(figsize = (12,6))

fig = sns.barplot(
    data=unemployment_rate_df, 
    x="Date", 
    y="Unemployment Rate",
    ax=ax
)

x_dates = unemployment_rate_df['Date'].dt.strftime('%Y-%m-%d').sort_values().unique()
arr = ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')


In [ ]:
cursor = conn.cursor();
cursor.execute("""SELECT date_of_interest, avg(case_count) 
    FROM covid_cases  
    WHERE date_of_interest >= '2020-01-01' and date_of_interest < '2022-01-01'
    group by date_of_interest order by date_of_interest asc""")
covid_cases_data = cursor.fetchall()
cursor.close()


covid_cases_data_df = pd.DataFrame(data = covid_cases_data,
                                   columns=["Date",
                                            "Case Count"])

fig, ax = plt.subplots(figsize = (12,6))
fig = sns.lineplot(data = covid_cases_data_df, 
                   x="Date", 
                   y="Case Count",
                   ax=ax
                  )



In [ ]:
cursor = conn.cursor();
query = """
SELECT date_of_interest, avg(data_value) 
    FROM air_quality
    WHERE date_of_interest >= '2020-01-01' and date_of_interest < '2022-01-01'
    GROUP BY  date_of_interest order by date_of_interest asc
"""
cursor.execute(query)
air_quality_data = cursor.fetchall()
cursor.close()


air_quality_data_df = pd.DataFrame(data = air_quality_data,
                                   columns=["Date",
                                            "Air Auality Data Value"])

# print(air_quality_data_df)
fig, ax = plt.subplots(figsize = (12,6))

fig = sns.scatterplot(data = air_quality_data_df, x="Date", y="Air Auality Data Value", ax=ax)


In [ ]:
unemployment_rate_df.head()

In [ ]:
covid_cases_data_df.head()

In [ ]:
air_quality_data_df.head()

In [ ]:
quality_covid_cases_air_quality_df = pd.merge(air_quality_data_df, covid_cases_data_df, how="outer", on ="Date")
all_data = pd.merge(quality_covid_cases_air_quality_df, unemployment_rate_df, how="left", on ="Date")

all_data["Air Auality Data Value"] = all_data["Air Auality Data Value"].fillna(0)
all_data["Covid Case Count"] = all_data["Case Count"].fillna(0)
all_data["Unemployment Rate"] = all_data["Unemployment Rate"].fillna(0)

print(all_data)


In [ ]:
ig, ax = plt.subplots(figsize = (12,6))

fig = sns.scatterplot(data = all_data, 
                      x="Date", 
                      y="Unemployment Rate", 
                      hue="Covid Case Count", 
                      size="Air Auality Data Value", ax=ax)


x_dates = unemployment_rate_df['Date'].dt.strftime('%Y-%m-%d').sort_values().unique()
arr = ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')


In [ ]:
bar_data = go.Bar(
          x = "Covid Case Count",
          y = "Unemployment Rate",
          name = 'Total Sales',
          yaxis = 'y1'
)

line_data = go.Bar(
          x = [2, 5, 7],
          y = [40, 50, 60],
          yaxis = 'y2'
) 



data = [bar_data, line_data]

layout = go.Layout(
    barmode='group',
    title = 'Total sales and quantity ordered',
    hovermode = 'closest',
    
    xaxis=dict(title='Name of Product',
                showline=True,
                showgrid=False,
                showticklabels=True,
                linecolor='rgb(104, 204, 104)',
                linewidth=2,
                ticks='outside',
         tickfont=dict(
                family='Arial',
                size=12,
                color='rgb(82, 82, 82)',
        ),
               
    ),
    yaxis=dict(title='Total Sales'),
    yaxis2=dict(title='Total quantity ordered', side='right',overlaying='y'),
    
     legend=dict(
        orientation='v',
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)',
        x=0.79,
        y=0.97,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    )
    
)

figure2 = go.Figure(data=data, layout=layout)
figure2.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
figure2.update_layout(legend_title='<b> Trend </b>')

figure2.update_layout(
    title_font_family="Times New Roman",
    title_font_color="red",
    title_font_size=30,
    legend_title_font_color="green")

figure2.update_layout(
    title={
        'text': 'Total sales and quantity ordered',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


figure2.show()